In [21]:
import geopandas as gpd
from scipy.spatial import KDTree

## Pull Landsat Row/Path from Landsat and Clip to US

In [11]:
def process_geometry(geom, study_area):
    """
    Reproject geometry to study area projection and clip to study area. Reset index because features are dropped during clipping.
    """
    geom.to_crs(study_area.crs, inplace=True)
    geom = gpd.clip(geom, study_area)
    geom.reset_index(inplace=True, drop=True)
    return geom


us_states_shp = gpd.read_file(
    "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip"
)[["geometry", "STUSPS"]]
study_area = us_states_shp[
    us_states_shp["STUSPS"].isin(["AK", "HI", "PR", "VI", "GU", "AS", "MP", "UM"])
    == False
]  # Remove non-continental US states

landsat_scenes_shp = gpd.read_file(
    "https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/WRS2_descending_0.zip"
)[["PATH", "ROW", "geometry"]]
landsat_scenes_shp_clipped = process_geometry(landsat_scenes_shp, study_area)

## Visualize Landsat Scenes Geographically

In [12]:
landsat_scenes_shp_clipped.explore()

# You can see the Columns (PATH) start around 48 and the Rows (ROW) start around 26.
# Each cell represents a scene that was captured by Landsat (180x180km).
# There are about 500 scenes in the continental US.

## Extract Path/Rows and put into a KDTree for nearest neighbor search

In [22]:
path_row_values = landsat_scenes_shp_clipped[["PATH", "ROW"]].values
path_row_values[:5]

array([[42, 37],
       [41, 37],
       [42, 36],
       [41, 36],
       [43, 36]])

In [28]:
tree = KDTree(path_row_values)
distances, indices = tree.query(
    [30, 32], k=5
)  # Find the 5 closest scenes to the path/row (30, 32) which is in the center of the continental US
# Visualize the 5 closest scenes to the point (100, 35)
landsat_scenes_shp_clipped.iloc[indices].explore()